 ## <center>Exploratory Data Analysis</center>
### <center>Distributions</center>
#### <center> Recommended Source: <a href="https://www.amazon.com/Think-Stats-Exploratory-Data-Analysis/dp/1491907339/ref=sr_1_1?crid=S92SQL92872G&keywords=THINK+STATS&qid=1668390828&sprefix=think+stats%2Caps%2C160&sr=8-1">Link</a></center>

For this guide I will be using my own

While analyzing data one of the best ways of describing a variable is to report values in the dataset and how many times each value appears, in other words view the Distribution of the variable. The most common representation is a <a href="https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/histograms-intro">"histogram"</a>, which is a graph that shows the frequency of each  value.

In [7]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv
import re
from sqlalchemy import create_engine

In [8]:
def process_samsung_files(csv_list):
    curated_csv_list = []
    for csv in csv_list:
        csv_curated = re.sub(r'[0-9]+', '', csv)
        csv_curated = csv_curated.replace("com.samsung.shealth.", "").replace("com.samsung.health.", "").replace(".csv", "").replace(".", "_")
        csv_curated = csv_curated.strip('_')
        accepted_csvs = [
            'tracker_pedometer_step_count','sleep_combined','sleep_stage', 'activity_day_summary', 'calories_burned_details', 'exercise', 'weight', 'preferences', 'sleep', 'step_daily_trend', 'stress', 'stress_histogram', 'tracker_heart_rate', 'tracker_oxygen_saturation', 'tracker.pedometer_day_summary', 'tracker_pedometer_step_count']
        if csv_curated in accepted_csvs:
            curated_csv_list.append(csv_curated)
            os.rename(
                os.path.join(os.path.join('datasets','watch_data_dump'), csv),
                os.path.join(os.path.join('datasets','watch_data_dump'), csv_curated)+".csv"
            )
        else:
            os.remove(os.path.join(os.path.join('datasets','watch_data_dump'), csv))

    return curated_csv_list
csv_list = process_samsung_files(
        os.listdir(
                os.path.join('datasets','watch_data_dump')
            )
    )

In [9]:
def clean_folder(path):

    csv_list = os.listdir(
                path
            )

    for file in csv_list:
        os.remove(os.path.join(path, file))


In [10]:
def get_watch_data():
    csv_list = process_samsung_files(
        os.listdir(
                os.path.join('datasets','watch_data_dump')
            )
    )
    watch_data_dict = {}
    for csv in csv_list:

        data = pd.read_csv(os.path.join(os.path.join('datasets','watch_data_dump'), csv+'.csv'), index_col=False, delimiter=',', skiprows=1)

        approved_att_list = [column.replace('com.samsung.shealth.', '').replace('com.samsung.health.', '') for column in data.columns if column not in [
            "goal",
            "others_time",
            "",
            "exercise.pkg_name",
            "exercise.exercise_custom_type",
            "exercise.datauuid",
            "exercise.sweat_loss",
            "exercise.pkg_name",
            "exercise.live_data",
            "exercise.comment",
            "exercise.deviceuuid",
            "exercise.additional",
            "exercise.location_data",
            "additional_internal",
            "location_data_internal",
            "heart_rate_deviceuuid",
            "program_schedule_id",
            "mission_extra_value",
            "mission_type",
            "tracking_status",
            "program_id",
            "title",
            "pace_live_data",
            "pace_info_id",
            "completion_status",
            "subset_data",
            "mission_value",
            "sleep_id",
            "create_time",
            "update_time",
            "algorithm",
            "comment",
            "heart_rate.comment",
            "heart_rate.pkg_name",
            "heart_rate.time_offset",
            "heart_rate.datauuid",
            "heart_rate.update_time",
            "heart_rate.create_time",
            "heart_rate.binning_data",
            "heart_rate.custom",
            "heart_rate.heart_beat_count",
            "oxygen_saturation.custom",
            "oxygen_saturation.create_time",
            "oxygen_saturation.update_time",
            "oxygen_saturation.binning",
            "oxygen_saturation.time_offset",
            "oxygen_saturation.deviceuuid",
            "oxygen_saturation.comment",
            "oxygen_saturation.pkg_name",
            "oxygen_saturation.datauuid",
            "version_code",
            "step_count.update_time",
            "step_count.create_time",
            "step_count.time_offset",
            "vfa_level",
            "comment",
            "time_offset",
            "muscle_mass",
            "heart_rate.deviceuuid",
            "heart_rate.comment",
            "heart_rate.pkg_name",
            "heart_rate.datauuid",
            "custom",
            "deviceuuid",
            "pkg_name",
            "datauuid",
            "extra_data",
            "device_type",
            "source",
            "source_id",
            "source_pkg_name",
            "com.samsung.shealth.calories_burned.pkg_name",
            "com.samsung.shealth.calories_burned.datauuid",
            "com.samsung.shealth.calories_burned.deviceuuid",
            "com.samsung.shealth.calories_burned.deviceuuid",
            "com.samsung.health.sleep.datauuid",
            "com.samsung.health.sleep.pkg_name",
            "com.samsung.health.sleep.deviceuuid",
            "com.samsung.health.sleep.comment",
            "binning_data",
            "tag_id",
            "source_type",
            "com.samsung.health.step_count.datauuid",
            "com.samsung.health.step_count.pkg_name",
            "com.samsung.health.step_count.deviceuuid",
            "com.samsung.health.step_count.sample_position_type",
            "com.samsung.health.step_count.custom",
        ] ]
        data.columns = data.columns.str.replace('com.samsung.shealth.', '', regex=False)
        data.columns = data.columns.str.replace('com.samsung.health.', '', regex=False)
        for column in data.columns:
            if "start_time" in column or "end_time" in column:
                data[column] = pd.to_datetime(data[column])
            elif "day_time" in column:
                data[column] = pd.to_datetime(data[column], unit='ms')
            elif "active" in column:
                data[column] = data[column]/1000/60
        for column in data.columns:

            if "start_time" in column:
                data = data[(data[column] >= '11/14/2022')]
                break

        data['user_id'] = 1
        data.fillna("", inplace=True)

        watch_data_dict[csv] = data[approved_att_list]
    clean_folder(os.path.join('datasets','watch_data_dump'))
    return watch_data_dict
watch_data = get_watch_data()


In [11]:
watch_data.keys()

dict_keys(['activity_day_summary', 'calories_burned_details', 'exercise', 'preferences', 'sleep', 'sleep_combined', 'sleep_stage', 'step_daily_trend', 'stress', 'stress_histogram', 'tracker_heart_rate', 'tracker_oxygen_saturation', 'tracker_pedometer_step_count', 'weight'])

In [12]:
watch_data['tracker_pedometer_step_count']


,duration,run_step,walk_step,step_count.start_time,step_count.update_time,step_count.create_time,step_count.count,step_count.speed,step_count.distance,step_count.calorie,step_count.time_offset,step_count.end_time
233,6984,0,18,2022-11-14 00:09:00,2022-11-14 00:09:40.901,2022-11-14 00:09:40.901,18,2.138889,14.940000,1.070000,UTC-0600,2022-11-14 00:10:00
238,9553,0,15,2022-11-14 01:34:00,2022-11-14 01:34:15.723,2022-11-14 01:34:15.723,15,1.138889,10.880000,1.050000,UTC-0600,2022-11-14 01:35:00
240,10049,0,14,2022-11-14 05:35:00,2022-11-14 05:36:31.473,2022-11-14 05:36:31.473,14,0.972222,9.770000,1.020000,UTC-0600,2022-11-14 05:36:00
242,6346,0,19,2022-11-14 00:11:00,2022-11-14 00:14:59.493,2022-11-14 00:14:59.493,19,2.472222,15.690000,1.170000,UTC-0600,2022-11-14 00:12:00
246,8812,0,16,2022-11-14 15:37:00,2022-11-14 15:37:19.648,2022-11-14 15:37:19.504,16,1.362859,12.010254,1.140015,UTC-0600,2022-11-14 15:38:00
...,...,...,...,...,...,...,...,...,...,...,...,...
549,4589,9,10,2022-11-18 21:42:00,2022-11-18 21:49:18.038,2022-11-18 21:49:18.038,19,3.861111,17.720000,2.250000,UTC-0600,2022-11-18 21:43:00
550,9855,0,8,2022-11-18 22:50:00,2022-11-18 22:50:18.981,2022-11-18 22:50:16.750,8,0.583215,5.747864,0.585526,UTC-0600,2022-11-18 22:51:00
551,18019,0,26,2022-11-19 02:42:00,2022-11-19 02:42:47.968,2022-11-19 02:42:29.751,26,1.077963,19.424316,1.858002,UTC-0600,2022-11-19 02:43:00
552,6569,0,10,2022-11-19 15:45:00,2022-11-19 15:45:19.079,2022-11-19 15:45:18.226,10,1.153249,7.575897,0.704186,UTC-0600,2022-11-19 15:46:00


In [13]:
def get_scale_data():
    csv_list = os.listdir(os.path.join('datasets','digital_scale_data_dump'))

    data = pd.DataFrame()
    for csv in csv_list:

        data = pd.read_csv(os.path.join(os.path.join('datasets','digital_scale_data_dump'), csv), delimiter=',')
        data['user_id'] = 1
    clean_folder(os.path.join('datasets','digital_scale_data_dump'))
    return data
scale_data = get_scale_data()
scale_data

,Time of Measurement,Weight(lb),BMI,Body Fat(%),Fat-free Body Weight(lb),Subcutaneous Fat(%),Visceral Fat,Body Water(%),Skeletal Muscle(%),Muscle Mass(lb),Bone Mass(lb),Protein(%),BMR(kcal),Metabolic Age,Remarks,user_id
0,2022-11-17 09:12:11,287.4,43.1,26.2,212.2,19.8,24.0,53.3,47.5,201.6,10.6,16.8,2454,39,NaN,1
1,2022-11-18 19:42:41,286.8,43.0,26.1,212.0,19.8,24.0,53.4,47.6,201.4,10.6,16.9,2445,39,NaN,1
2,2022-11-19 11:39:43,288.2,43.2,26.2,212.6,19.8,24.0,53.3,47.5,202.0,10.6,16.8,2458,39,NaN,1


### Centralizing Data

I first start by connecting to my ibm db2 database in order to push my data and start working

In [14]:
load_dotenv()
usr = os.getenv("db_usr")
pwd = os.getenv("db_pwd")
hostname = os.getenv("db_hostname")
db_name = os.getenv("db_name")

usr,pwd,hostname,db_name

('aycahfkimgybra',
 'ca1349f5d7c74eaa0b8191cdb05e84ead5f4046ac49e47422738c59668932f69',
 'ec2-18-214-35-70.compute-1.amazonaws.com',
 'dqdnv4gccha9p')

In [17]:
!pip install psycopg2
connection_uri = f'postgresql+psycopg2://{usr}:{pwd}@{hostname}/{db_name}'
sqlEngine = create_engine(connection_uri)

conn    = sqlEngine.connect()

     ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00


In [ ]:
forbidden = ['preferences', 'stress_histogram']
for key in watch_data.keys():

    if key.replace('.','_') not in forbidden:
        watch_data[key].to_sql(key.replace('.','_'), con=conn, if_exists='replace', index=False)

stages_data_mapping = {'id':[4001, 4002, 4003, 4004], "stages": ['LIGHT', 'DEEP', 'REM', 'AWAKE']}
sleep_stages = pd.DataFrame(stages_data_mapping)
sleep_stages.to_sql('sleep_stages_mapping', con=conn, if_exists='replace', index=False)

In [ ]:
try:
    scale_data_stored = pd.read_sql('scale_data',conn)
    result = scale_data_stored.append(scale_data)
    result.drop_duplicates(keep='last', inplace=True)
    result.to_sql('scale_data', con=conn, if_exists='append', index=False)
except:
    scale_data.to_sql('scale_data', con=conn, if_exists='append', index=False)

### LOADING DATA

In [ ]:
240120/1000/60